In [331]:
import sys
import logging
from uuid import UUID

import emission.core.get_database as edb
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as pydt
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.timequery as estt

logging.getLogger().setLevel(logging.DEBUG)
logging.debug(sys.path)

iphone_ids = [UUID("079e0f1a-c440-3d7c-b0e7-de160f748e35"), UUID("c76a0487-7e5a-3b17-a449-47be666b36f6"), 
              UUID("c528bcd2-a88b-3e82-be62-ef4f2396967a")]
android_ids = [UUID("e471711e-bd14-3dbe-80b6-9c7d92ecc296"), UUID("fd7b4c2e-2c8b-3bfa-94f0-d1e3ecbd5fb7"),
               UUID("86842c35-da28-32ed-a90e-2da6663c5c73")]

In [332]:
list(edb.get_timeseries_db().find({"metadata.key": "config/sensor_config", "data.battery_status": {"$exists": True}}).sort("metadata.write_ts").limit(10))

[]

In [333]:
#type: list of emission.storage.timeseries.builtin_timeseries.BuiltinTimeSeries
iphone_ts = map(lambda u: esta.TimeSeries.get_time_series(u), iphone_ids)
android_ts = map(lambda u: esta.TimeSeries.get_time_series(u), android_ids)

#time query specification
dt1 = pydt.datetime(2016, 6, 28, 0, 0) #datetime.datetime(2016, 6, 24, 22, 0)
dt2 = pydt.datetime(2016, 6, 29, 8, 0)

ts1 = int(dt1.strftime("%s"))
ts2 = int(dt2.strftime("%s"))

time_range = estt.TimeQuery("metadata.write_ts", ts1, ts2)

#BuiltinTimeSeries.get_data_df(key, time_query = None, geo_query = None, extra_query_list=None) 
#type: list of DataFrame objects (regarding state info)
iphone_state_df_list = map(lambda t:t.get_data_df("statemachine/transition", time_range), iphone_ts)
android_state_df_list = map(lambda t:t.get_data_df("statemachine/transition", time_range), android_ts)

iphone_battery_df_list = map(lambda t:t.get_data_df("background/battery", time_range), iphone_ts)
android_battery_df_list = map(lambda t:t.get_data_df("background/battery", time_range), android_ts)

In [334]:
_iphone_config_df_list = map(lambda t:t.get_data_df("config/sensor_config", time_range), iphone_ts)
_android_config_df_list = map(lambda t:t.get_data_df("config/sensor_config", time_range), android_ts)

ts_to_dt = lambda t: pydt.datetime.fromtimestamp(t)

iphone_config_df_list=[]
for i, df in enumerate(_iphone_config_df_list):
    if len(df)<>0:
        df['config_time'] = df['metadata_write_ts'].apply(ts_to_dt)
        #df['ts'] = df['metadata_write_ts']
        iphone_config_df_list.append(df)
    else:
        iphone_config_df_list.append(-1)
        print "no config log for ios" + str(i)

android_config_df_list=[]
for i, df in enumerate(_android_config_df_list):
    if len(df)<>0:
        df['config_time'] = df['metadata_write_ts'].apply(ts_to_dt)
        #df['ts'] = df['metadata_write_ts'] # <class 'pandas.core.series.Series'>
        android_config_df_list.append(df)
    else:
        android_config_df_list.append(-1)
        print "no config log for android" + str(i)

no config log for ios2
no config log for android1
no config log for android2


In [335]:
def config_changed(df):
    changed_lst = ['Null']
    temp = df.copy()
    temp = temp.drop(['_id', 'metadata_write_ts', 'local_dt', 'ts'], 1)
    #print temp
    #temp.reset_index(inplace=True)
    for i in range(1, len(temp)):
        #print(temp.iloc[i]==temp.iloc[i-1])
        if temp.iloc[i].equals(temp.iloc[i-1]):
            changed_lst.append('0')
        else:
            changed_lst.append('1')
    return changed_lst

In [345]:
iphone_config_change_df_list=[]
for index, df in enumerate(iphone_config_df_list):
    print type(df)
    
for index, df in enumerate(iphone_config_df_list):
    print type(df)
    print "done"
    if df==-1:
        #no config log found 
        iphone_config_change_df_list.append(-1)
    else:
        config_series = config_changed(df)
        df.loc[:, 'config_changed'] = pd.Series(config_series, index=df.index)
        iphone_config_change_df_list.append(df)
    
android_config_change_df_list=[]
for index, df in enumerate(android_config_df_list):
    if df==-1:
        #no config log found 
        android_config_change_df_list.append(-1)
    else:
        config_series = config_changed(df)
        df.loc[:, 'config_changed'] = pd.Series(config_series, index=df.index)
        android_config_change_df_list.append(df)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<type 'int'>
<class 'pandas.core.frame.DataFrame'>
done


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [328]:
ios1 = iphone_config_change_df_list[0]
ios1
android1 = android_config_change_df_list[0]
android1

IndexError: list index out of range

In [329]:
#checking Android data...
n = 0

a_s = android_state_df_list[n][['metadata_write_ts', 'fmt_time', 'curr_state', 'transition']]
a_s.rename(columns = {'fmt_time':'state_time'}, inplace = True)

a_b = android_battery_df_list[n][['metadata_write_ts', 'fmt_time', 'battery_level_pct', 'battery_status']]
a_b.rename(columns = {'fmt_time':'battery_time'}, inplace = True)

#a_c = android_config_change_df_list[n]

if a_c <> -1: 
    a_sc = pd.merge(a_s, a_c, on='metadata_write_ts', how='outer')
    a_sc.sort_values('metadata_write_ts', ascending=True, inplace=True)
    #ts_to_dt = lambda t: pydt.datetime.fromtimestamp(t)
    #a_sc['dt']=a_sc['metadata_write_ts'].apply(ts_to_dt)

    #print len(a_combined)
    a_sc.fillna('-')[['state_time','config_time', 'curr_state', 'transition', 'config_changed']]

    a_all = pd.merge(a_combined, a_b, on='metadata_write_ts', how='outer')
    a_all.sort_values('metadata_write_ts', ascending=True, inplace=True)
    a_all.fillna('-')[['state_time','config_time', 'battery_time','curr_state', 'transition', 'config_changed', 'battery_level_pct', 'battery_status']]

a_sb = pd.merge(a_s, a_b, on='metadata_write_ts', how='outer')
a_sb.sort_values('metadata_write_ts', ascending=True, inplace=True)
a_sb.fillna('-')[['state_time', 'battery_time','curr_state', 'transition', 'battery_level_pct', 'battery_status']]


,state_time,battery_time,curr_state,transition,battery_level_pct,battery_status
6,-,2016-06-28T00:07:29.438000-07:00,-,-,100,3
7,-,2016-06-28T00:17:29.508000-07:00,-,-,100,3
0,2016-06-28T00:27:16.846000-07:00,-,3,3,-,-
8,-,2016-06-28T00:37:31.302000-07:00,-,-,100,1
9,-,2016-06-28T00:47:31.043000-07:00,-,-,100,1
10,-,2016-06-28T00:57:31.195000-07:00,-,-,100,1
11,-,2016-06-28T01:07:30.955000-07:00,-,-,100,1
12,-,2016-06-28T01:17:30.578000-07:00,-,-,100,1
13,-,2016-06-28T01:37:30.717000-07:00,-,-,100,1
14,-,2016-06-28T02:32:13.976000-07:00,-,-,99,1


In [330]:
#checking iOS data...
n = 0
ios_s = iphone_state_df_list[n][['metadata_write_ts', 'fmt_time', 'curr_state', 'transition']]
ios_s.rename(columns = {'fmt_time':'state_time'}, inplace = True)

ios_b = iphone_battery_df_list[n][['metadata_write_ts', 'fmt_time', 'battery_level_pct', 'battery_status']]
ios_b.rename(columns = {'fmt_time':'battery_time'}, inplace = True)

ios_c = iphone_config_change_df_list[n]

if ios_c <> -1:
    ios_sc = pd.merge(ios_s_new, ios_c, on='metadata_write_ts', how='outer')
    ios_sc.sort_values('metadata_write_ts', ascending=True, inplace=True)
    #ts_to_dt = lambda t: pydt.datetime.fromtimestamp(t)
    #ios_sc['dt']=ios_sc['metadata_write_ts'].apply(ts_to_dt)

    #print len(ios_sc)
    #ios_sc.fillna('-')[['state_time','config_time','curr_state', 'transition', 'config_changed', 'ios_use_visit_notifications_for_detection']]

ios_s_new = ios_s[(ios_s.transition!=7) & (ios_s.transition!=16)]
ios_sb = pd.merge(ios_s_new, ios_b, on='metadata_write_ts', how='outer')
ios_sb.sort_values('metadata_write_ts', ascending=True, inplace=True)
ios_sb.fillna('-')[['state_time', 'battery_time','curr_state', 'transition', 'battery_level_pct', 'battery_status']]
ios_b

,metadata_write_ts,battery_time,battery_level_pct,battery_status
0,1.467097e+09,2016-06-28T00:00:04.321341-07:00,100.0,3
1,1.467097e+09,2016-06-28T00:00:17.327839-07:00,100.0,3
2,1.467097e+09,2016-06-28T00:00:20.005180-07:00,100.0,3
3,1.467097e+09,2016-06-28T00:00:32.598396-07:00,100.0,3
4,1.467098e+09,2016-06-28T00:10:03.418114-07:00,100.0,3
5,1.467098e+09,2016-06-28T00:10:05.921160-07:00,100.0,3
6,1.467098e+09,2016-06-28T00:20:04.512783-07:00,100.0,3
7,1.467098e+09,2016-06-28T00:20:07.297982-07:00,100.0,3
8,1.467099e+09,2016-06-28T00:25:00.882223-07:00,100.0,3
9,1.467099e+09,2016-06-28T00:25:00.967519-07:00,100.0,3
